In [1]:
import numpy as np
import pandas as pd

## GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'data1' : np.random.randn(5),
                  'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.572213,0.246974
1,a,two,1.499159,-1.420619
2,b,one,1.174149,1.099072
3,b,two,0.726690,0.212227
4,a,one,-0.541682,-0.306315


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped # It is an groupby object

In [7]:
grouped.mean()

key1
a    0.128421
b    0.950420
Name: data1, dtype: float64

In [11]:
# multiple arrays as index
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [12]:
means

key1  key2
a     one    -0.556948
      two     1.499159
b     one     1.174149
      two     0.726690
Name: data1, dtype: float64

In [13]:
means.unstack()

key2,one,two
key1,,
a,-0.556948,1.499159
b,1.174149,0.726690


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.556948 -0.029671
     two   1.499159 -1.420619
b    one   1.174149  1.099072
     two   0.726690  0.212227

### Iterating Over Groups

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.572213  0.246974
1    a  two  1.499159 -1.420619
4    a  one -0.541682 -0.306315
b
  key1 key2     data1     data2
2    b  one  1.174149  1.099072
3    b  two  0.726690  0.212227


In [16]:
pieces = dict(list(df.groupby('key1')))

In [17]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.572213  0.246974
 1    a  two  1.499159 -1.420619
 4    a  one -0.541682 -0.306315,
 'b':   key1 key2     data1     data2
 2    b  one  1.174149  1.099072
 3    b  two  0.726690  0.212227}

### Selecting a Column or Subset of Columns

In [25]:
df['data2'].groupby(df['key1'])

In [26]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.029671
      two    -1.420619
b     one     1.099072
      two     0.212227
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [27]:
people = pd.DataFrame(np.random.randn(5, 5),
....: columns=['a', 'b', 'c', 'd', 'e'],
....: index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [28]:
people

,a,b,c,d,e
Joe,0.223439,-1.664558,-1.286458,0.269851,0.681717
Steve,-0.151817,-0.167434,-1.097284,0.140485,0.720514
Wes,0.210866,-0.775003,0.552645,-1.663562,-0.156136
Jim,-0.855101,-0.477703,0.471676,1.676207,-0.049069
Travis,-0.306014,-0.403388,-0.747097,0.815947,-1.207746


In [34]:
people.iloc[2:3, [1,2]] = np.nan

In [36]:
people

,a,b,c,d,e
Joe,0.223439,-1.664558,-1.286458,0.269851,0.681717
Steve,-0.151817,-0.167434,-1.097284,0.140485,0.720514
Wes,0.210866,NaN,NaN,-1.663562,-0.156136
Jim,-0.855101,-0.477703,0.471676,1.676207,-0.049069
Travis,-0.306014,-0.403388,-0.747097,0.815947,-1.207746


In [39]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
....: 'd': 'blue', 'e': 'red', 'f' : 'orange'} # unused grouping keys are ok: like 'f'

In [40]:
by_column = people.groupby(mapping, axis = 1)

In [42]:
by_column.sum()

,blue,red
Joe,-1.016606,-0.759402
Steve,-0.956799,0.401263
Wes,-1.663562,0.054729
Jim,2.147884,-1.381873
Travis,0.068850,-1.917148


In [43]:
map_series = pd.Series(mapping)

In [44]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [45]:
people.groupby(map_series,axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [46]:
# group by the length of name
people.groupby(len).sum()

,a,b,c,d,e
3,-0.420797,-2.142260,-0.814781,0.282497,0.476512
5,-0.151817,-0.167434,-1.097284,0.140485,0.720514
6,-0.306014,-0.403388,-0.747097,0.815947,-1.207746


In [47]:
# ?
key_list = ['one', 'one', 'one', 'two', 'two']

In [48]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.210866 -1.664558 -1.286458 -1.663562 -0.156136
  two -0.855101 -0.477703  0.471676  1.676207 -0.049069
5 one -0.151817 -0.167434 -1.097284  0.140485  0.720514
6 two -0.306014 -0.403388 -0.747097  0.815947 -1.207746

### Grouping by Index Levels

In [54]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
....: [1, 3, 5, 1, 3]],
....: names=['city', 'tenor'])

In [55]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

In [56]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -1.074207  0.178064 -0.726523 -1.608654 -0.032650
1     -1.292835  0.723032 -0.471061 -0.648659  0.552537
2      0.756912  0.913154  0.909875 -2.286968 -0.345278
3     -0.475928 -0.918558 -0.567181  1.010378 -1.024247

In [57]:
hier_df.groupby(level = 'city', axis = 1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [58]:
def peak_to_peak (arr):
    return arr.max() - arr.min()

In [60]:
grouped = df.groupby('key1')

In [61]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.071372,1.667592
b,0.447459,0.886845


In [62]:
group.describe()

,data1,data2
count,2.000000,2.000000
mean,0.950420,0.655649
std,0.316401,0.627094
min,0.726690,0.212227
25%,0.838555,0.433938
50%,0.950420,0.655649
75%,1.062284,0.877361
max,1.174149,1.099072


### Column-Wise and Multiple Function Application

In [63]:
tips = pd.read_csv('tips.csv')

In [64]:
tips[:6]

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4


In [69]:
grouped = tips.groupby(['day', 'smoker'])

In [70]:
grouped_tip = grouped['tip'] 


In [72]:
grouped_tip.agg('mean')

day   smoker
Fri   No        2.812500
      Yes       2.714000
Sat   No        3.102889
      Yes       2.875476
Sun   No        3.167895
      Yes       3.516842
Thur  No        2.673778
      Yes       3.030000
Name: tip, dtype: float64

In [73]:
grouped_tip.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      2.812500  0.898494          2.00
     Yes     2.714000  1.077668          3.73
Sat  No      3.102889  1.642088          8.00
     Yes     2.875476  1.630580          9.00
Sun  No      3.167895  1.224785          4.99
     Yes     3.516842  1.261151          5.00
Thur No      2.673778  1.282964          5.45
     Yes     3.030000  1.113491          3.00

In [74]:
# if you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names 
grouped_tip.agg([('fool', 'mean'), ('bar', np.std)])

fool       bar
day  smoker                    
Fri  No      2.812500  0.898494
     Yes     2.714000  1.077668
Sat  No      3.102889  1.642088
     Yes     2.875476  1.630580
Sun  No      3.167895  1.224785
     Yes     3.516842  1.261151
Thur No      2.673778  1.282964
     Yes     3.030000  1.113491

In [75]:
functions = ['count', 'mean', 'max']

In [76]:
result = grouped['tip', 'total_bill'].agg(functions)

In [77]:
result

tip                  total_bill                  
            count      mean    max      count       mean    max
day  smoker                                                    
Fri  No         4  2.812500   3.50          4  18.420000  22.75
     Yes       15  2.714000   4.73         15  16.813333  40.17
Sat  No        45  3.102889   9.00         45  19.661778  48.33
     Yes       42  2.875476  10.00         42  21.276667  50.81
Sun  No        57  3.167895   6.00         57  20.506667  48.17
     Yes       19  3.516842   6.50         19  24.120000  45.35
Thur No        45  2.673778   6.70         45  17.113111  41.19
     Yes       17  3.030000   5.00         17  19.190588  43.11

In [80]:
ftuples = [('均值', 'mean'), ('Abweichung', np.var)]

In [81]:
grouped['tip', 'total_bill'].agg(ftuples)

tip            total_bill            
                   均值 Abweichung         均值  Abweichung
day  smoker                                            
Fri  No      2.812500   0.807292  18.420000   25.596333
     Yes     2.714000   1.161369  16.813333   82.562438
Sat  No      3.102889   2.696453  19.661778   79.908965
     Yes     2.875476   2.658791  21.276667  101.387535
Sun  No      3.167895   1.500099  20.506667   66.099980
     Yes     3.516842   1.590501  24.120000  109.046044
Thur No      2.673778   1.645997  17.113111   59.625081
     Yes     3.030000   1.239863  19.190588   69.808518

In [84]:
grouped.agg({'tip' : ['min', 'max', 'mean', 'std'],'size' : 'sum'})

tip                            size
              min    max      mean       std  sum
day  smoker                                      
Fri  No      1.50   3.50  2.812500  0.898494    9
     Yes     1.00   4.73  2.714000  1.077668   31
Sat  No      1.00   9.00  3.102889  1.642088  115
     Yes     1.00  10.00  2.875476  1.630580  104
Sun  No      1.01   6.00  3.167895  1.224785  167
     Yes     1.50   6.50  3.516842  1.261151   49
Thur No      1.25   6.70  2.673778  1.282964  112
     Yes     2.00   5.00  3.030000  1.113491   40

### Returning Aggregated Data Without Row Indexes

In [85]:
tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size
0,Fri,No,18.420000,2.812500,2.250000
1,Fri,Yes,16.813333,2.714000,2.066667
2,Sat,No,19.661778,3.102889,2.555556
3,Sat,Yes,21.276667,2.875476,2.476190
4,Sun,No,20.506667,3.167895,2.929825
5,Sun,Yes,24.120000,3.516842,2.578947
6,Thur,No,17.113111,2.673778,2.488889
7,Thur,Yes,19.190588,3.030000,2.352941


## 10.3 Apply: General split-apply-combine

In [93]:
def top(df, n=5, column = 'tip'):
    return df.sort_values(by=column)[-n:]

In [94]:
top(tips, n = 6, column = 'tip')

,total_bill,tip,sex,smoker,day,time,size
214,28.17,6.50,Female,Yes,Sat,Dinner,3
141,34.30,6.70,Male,No,Thur,Lunch,6
59,48.27,6.73,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
212,48.33,9.00,Male,No,Sat,Dinner,4
170,50.81,10.00,Male,Yes,Sat,Dinner,3


In [99]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='tip')

total_bill    tip     sex smoker   day    time  size
smoker day                                                           
No     Fri  91        22.49   3.50    Male     No   Fri  Dinner     2
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4
       Sun  47        32.40   6.00    Male     No   Sun  Dinner     4
       Thur 141       34.30   6.70    Male     No  Thur   Lunch     6
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3
       Sun  183       23.17   6.50    Male    Yes   Sun  Dinner     4
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4

In [101]:
result = tips.groupby('smoker')['tip'].describe()

In [102]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,2.991854,1.377190,1.0,2.0,2.74,3.505,9.0
Yes,93.0,3.008710,1.401468,1.0,2.0,3.00,3.680,10.0


In [103]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          2.991854
       Yes         3.008710
std    No          1.377190
       Yes         1.401468
min    No          1.000000
       Yes         1.000000
25%    No          2.000000
       Yes         2.000000
50%    No          2.740000
       Yes         3.000000
75%    No          3.505000
       Yes         3.680000
max    No          9.000000
       Yes        10.000000
dtype: float64

In [104]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size
47,32.40,6.00,Male,No,Sun,Dinner,4
141,34.30,6.70,Male,No,Thur,Lunch,6
59,48.27,6.73,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
212,48.33,9.00,Male,No,Sat,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
181,23.33,5.65,Male,Yes,Sun,Dinner,2
214,28.17,6.50,Female,Yes,Sat,Dinner,3
183,23.17,6.50,Male,Yes,Sun,Dinner,4
170,50.81,10.00,Male,Yes,Sat,Dinner,3


### Quantile and Bucket Analysis

In [105]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
....: 'data2': np.random.randn(1000)})

In [106]:
quartiles = pd.cut(frame.data1, 4)

In [108]:
def get_stats(group):
    return {'min': group.min(), 'max' : group.max(), 'count': group.count(), 'mean': group.mean()}

In [109]:
grouped = frame.data2.groupby(quartiles)

In [110]:
grouped

In [111]:
grouped.apply(get_stats)  # ??

data1                  
(-3.264, -1.717]  count     41.000000
                  max        1.534371
                  mean      -0.193889
                  min       -1.595233
(-1.717, -0.177]  count    380.000000
                  max        2.883321
                  mean       0.082291
                  min       -2.987831
(-0.177, 1.364]   count    492.000000
                  max        2.666984
                  mean      -0.022954
                  min       -2.913228
(1.364, 2.904]    count     87.000000
                  max        2.723959
                  mean       0.090889
                  min       -2.974641
Name: data2, dtype: float64

In [112]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [113]:
grouped = frame.data2.groupby(grouping)

In [115]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.442541,-0.017315,-1.658904
1,100.0,2.142596,-0.058544,-2.987831
2,100.0,2.883321,0.151223,-2.397110
3,100.0,2.717486,0.113683,-2.039874
4,100.0,2.727600,0.175123,-2.361792
5,100.0,1.852300,-0.148316,-2.739341
6,100.0,2.535261,-0.001197,-1.633774
7,100.0,2.291804,0.003182,-2.913228
8,100.0,1.997338,-0.057546,-2.528558


### Example: Filling Missing Values with Group-Specific Values

In [117]:
s = pd.Series(np.random.randn(6))

In [118]:
s[::2] = np.nan

In [119]:
s

0         NaN
1    0.137864
2         NaN
3    0.131327
4         NaN
5    0.212733
dtype: float64

In [120]:
s.fillna(s.mean())

0    0.160641
1    0.137864
2    0.160641
3    0.131327
4    0.160641
5    0.212733
dtype: float64

In [121]:
 states = ['Ohio', 'New York', 'Vermont', 'Florida',
....: 'Oregon', 'Nevada', 'California', 'Idaho']

In [122]:
group_key = ['East'] * 4 + ['West'] * 4

In [123]:
data = pd.Series(np.random.randn(8), index = states)

In [124]:
data

Ohio          0.227926
New York     -0.639068
Vermont      -1.144277
Florida      -0.179288
Oregon        0.155372
Nevada        0.615844
California    0.646427
Idaho         0.188333
dtype: float64

In [125]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [126]:
data

Ohio          0.227926
New York     -0.639068
Vermont            NaN
Florida      -0.179288
Oregon        0.155372
Nevada             NaN
California    0.646427
Idaho              NaN
dtype: float64

In [127]:
data.groupby(group_key).mean()

East   -0.196810
West    0.400899
dtype: float64

In [135]:
fill_mean = lambda g: g.fillna(g.mean())
fill_mean = lambda g: g.fillna(g.mean())

In [137]:
data.groupby(group_key).apply(fill_mean)
# data.groupby(group_key).apply(fill_mean)

Ohio          0.227926
New York     -0.639068
Vermont      -0.196810
Florida      -0.179288
Oregon        0.155372
Nevada        0.400899
California    0.646427
Idaho         0.400899
dtype: float64

In [138]:
fill_values = {'East': 0.5, 'West': -1}

In [144]:
# the groups have a name attribute set internally
fill_func = lambda g: g.fillna(fill_values[g.name])

In [145]:
data.groupby(group_key).apply(fill_func)
# data.groupby(group_key).apply(fill_func)

Ohio          0.227926
New York     -0.639068
Vermont       0.500000
Florida      -0.179288
Oregon        0.155372
Nevada       -1.000000
California    0.646427
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [146]:
suits = ['H', 'S', 'C', 'D']

In [147]:
card_val = (list(range(1,11)) + [10] * 3) * 4

In [148]:
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']

In [151]:
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

In [152]:
deck = pd.Series(card_val, index = cards)

In [153]:
deck.sample(5)

8S      8
9S      9
3S      3
JH     10
10S    10
dtype: int64

In [155]:
def draw(deck, n):
    return deck.sample(n)

In [156]:
get_suit = lambda card: card[-1]

In [158]:
deck.groupby(get_suit).apply(draw, n = 2)

C  8C      8
   10C    10
D  6D      6
   3D      3
H  10H    10
   JH     10
S  2S      2
   6S      6
dtype: int64

In [159]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC    10
KC    10
6D     6
7D     7
7H     7
AH     1
9S     9
QS    10
dtype: int64

### Example: Group Weighted Average and Correlation

In [160]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
.....: 'b', 'b', 'b', 'b'],
.....: 'data': np.random.randn(8),
.....: 'weights': np.random.rand(8)})

In [162]:
df

,category,data,weights
0,a,0.168968,0.827039
1,a,-0.106244,0.576475
2,a,1.376781,0.758869
3,a,0.834685,0.207859
4,b,-2.083979,0.190647
5,b,0.717596,0.965232
6,b,-0.970517,0.950664
7,b,2.600327,0.456273


In [164]:
grouped = df.groupby('category')

In [171]:
get_wavg = lambda g: np.average(g['data'], weights = g['weights'])

In [172]:
grouped.apply(get_wavg)

category
a    0.547113
b    0.218184
dtype: float64

In [177]:
close_px = pd.read_csv('stock_px.csv', parse_dates = True, index_col = 0)

In [178]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [188]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [189]:
res = close_px.pct_change().dropna()

In [190]:
get_year = lambda x: x.year

In [191]:
by_year = res.groupby(get_year)

In [192]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [193]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [194]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [196]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables and Cross-Tabulation

In [198]:
tips.pivot_table(index=['day', 'smoker', 'tip'])

size  total_bill
day  smoker tip                       
Fri  No     1.50  2.000000   12.460000
            3.00  3.000000   15.980000
            3.25  2.000000   22.750000
            3.50  2.000000   22.490000
     Yes    1.00  2.000000    5.750000
            1.50  2.000000   12.030000
            1.58  2.000000   13.420000
            1.92  1.000000    8.580000
            2.00  2.000000   10.090000
            2.20  2.000000   12.160000
            2.50  2.000000   13.810000
            3.00  2.000000   21.786667
            3.48  2.000000   13.420000
            4.00  2.000000   27.280000
            4.30  2.000000   16.320000
            4.73  4.000000   40.170000
Sat  No     1.00  1.000000    7.250000
            1.25  2.000000   10.290000
            1.45  2.000000    9.550000
            1.47  2.000000   10.770000
            1.50  2.000000   26.410000
            1.75  2.000000   17.820000
            1.97  2.000000   12.020000
            2.00  2.333333   14.123333
            2.01  2.000000   20.230000
            2.20  2.000000   14.730000
            2.23  2.000000   15.770000
            2.24  3.000000   16.040000
            2.31  3.000000   18.690000
            2.34  4.000000   17.810000
...                    ...         ...
Thur No     2.03  2.000000   15.980000
            2.18  3.000000   22.820000
            2.30  2.000000   16.430000
            2.31  2.000000   11.690000
            2.50  2.000000   14.260000
            2.52  2.000000   12.480000
            2.71  2.000000   17.290000
            2.75  2.000000   13.160000
            2.83  2.000000   20.270000
            2.92  4.000000   24.080000
            3.00  2.000000   20.900000
            3.25  2.000000   18.260000
            3.40  2.000000   16.660000
            3.50  2.000000   17.470000
            4.00  3.000000   22.740000
            4.20  6.000000   29.800000
            5.00  5.500000   34.120000
            5.17  4.000000   34.830000
            5.85  2.000000   24.710000
            6.70  6.000000   34.300000
     Yes    2.00  2.000000   13.170000
            2.01  2.000000   12.740000
            2.02  2.000000   15.480000
            2.50  2.000000   16.400000
            2.56  2.000000   28.440000
            3.00  2.000000   19.440000
            3.23  3.000000   16.470000
            4.00  3.000000   18.606667
            4.19  2.000000   19.810000
            5.00  3.000000   37.895000

[177 rows x 2 columns]

In [202]:
tips.pivot_table(['tip', 'size'], index = ['time', 'day'], columns = 'smoker')

size                 tip          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  2.750000  3.003333
       Sat   2.555556  2.476190  3.102889  2.875476
       Sun   2.929825  2.578947  3.167895  3.516842
       Thur  2.000000       NaN  3.000000       NaN
Lunch  Fri   3.000000  1.833333  3.000000  2.280000
       Thur  2.500000  2.352941  2.666364  3.030000

In [205]:
# margin??
tips.pivot_table(['tip', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                           tip                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  2.750000  3.003333  2.940000
       Sat   2.555556  2.476190  2.517241  3.102889  2.875476  2.993103
       Sun   2.929825  2.578947  2.842105  3.167895  3.516842  3.255132
       Thur  2.000000       NaN  2.000000  3.000000       NaN  3.000000
Lunch  Fri   3.000000  1.833333  2.000000  3.000000  2.280000  2.382857
       Thur  2.500000  2.352941  2.459016  2.666364  3.030000  2.767705
All          2.668874  2.408602  2.569672  2.991854  3.008710  2.998279

In [209]:
tips.pivot_table('tip', index=['time', 'smoker'], columns='day',aggfunc = len, margins = True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [211]:
tips.pivot_table('tip', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)


day                    Fri       Sat       Sun      Thur
time   size smoker                                      
Dinner 1    No      0.0000  1.000000  0.000000  0.000000
            Yes     0.0000  1.000000  0.000000  0.000000
       2    No      2.7500  2.563200  2.582222  3.000000
            Yes     2.7875  2.476786  3.345000  0.000000
       3    No      0.0000  3.412500  2.937273  0.000000
            Yes     0.0000  4.568333  3.625000  0.000000
       4    No      0.0000  4.800000  3.962500  0.000000
            Yes     4.7300  3.335000  5.090000  0.000000
       5    No      0.0000  0.000000  5.070000  0.000000
            Yes     0.0000  3.000000  2.000000  0.000000
       6    No      0.0000  0.000000  5.000000  0.000000
Lunch  1    No      0.0000  0.000000  0.000000  1.830000
            Yes     1.9200  0.000000  0.000000  0.000000
       2    No      0.0000  0.000000  0.000000  2.322353
            Yes     2.3520  0.000000  0.000000  2.713846
       3    No      3.0000  0.000000  0.000000  1.770000
            Yes     0.0000  0.000000  0.000000  3.615000
       4    No      0.0000  0.000000  0.000000  4.030000
            Yes     0.0000  0.000000  0.000000  4.500000
       5    No      0.0000  0.000000  0.000000  5.000000
       6    No      0.0000  0.000000  0.000000  5.300000

### Cross-Tabulations: Crosstab

In [212]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244